# IBM Data Science: Capstone Project
## Neighborhood Clustering
### Week 3 Assignment

## Part 1- Wikipedia Scraping

In [95]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [96]:
#Reading in the table and setting the header to the first row
df = pd.read_html(r'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header = 0)
df = pd.DataFrame(df[0])
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [97]:
#dropping rows with unassigned Boroughs
df = df[df.Borough != "Not assigned"]
df.reset_index(inplace = True, drop = True)
df.head()
#We can see that we have dropped 77 rows

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [98]:
#Replacing the "/" with a ","
df['Neighborhood'] = df['Neighborhood'].replace("/", ",", regex = True)
df.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


In [99]:
df_clean.shape

(103, 3)

## Part 2- Sourcing Latitude and Longitude

In [100]:
#Importing the CSV file with geo-loc data
geo = pd.read_csv(r'http://cocl.us/Geospatial_data', header = 0)
geo.shape

(103, 3)

In [101]:
#Taking a look at the schema
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [102]:
#Inner merging the two dataframes to get a consolidated data table
df_merged = df_clean.merge(geo, left_on = 'Postal code', right_on = 'Postal Code', sort = False)
df_merged.head()

,Postal code,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",M5A,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",M6A,43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",M7A,43.662301,-79.389494


In [103]:
#Dropping the duplicate postal code column after merge
df_merged.drop(['Postal code'], axis = 1, inplace = True)
df_merged.head()

,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,North York,Parkwoods,M3A,43.753259,-79.329656
1,North York,Victoria Village,M4A,43.725882,-79.315572
2,Downtown Toronto,"Regent Park , Harbourfront",M5A,43.654260,-79.360636
3,North York,"Lawrence Manor , Lawrence Heights",M6A,43.718518,-79.464763
4,Downtown Toronto,"Queen's Park , Ontario Provincial Government",M7A,43.662301,-79.389494


In [104]:
#Renaming the Postal Code field so it matches the assignment example
df_merged.rename(columns = {'Postal Code': 'PostalCode'}, inplace = True)
df_merged.columns

Index(['Borough', 'Neighborhood', 'PostalCode', 'Latitude', 'Longitude'], dtype='object')

In [105]:
#Ensuring that the ordering of the fields matches the example in the assignment
df_merged = df_merged[['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']]
df_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494


# Section 3: Clustering

In [132]:
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library

In [117]:
#Creating a subset of our data with boroughs that have the word "Toronto"
df_toronto = df_merged[df_merged['Borough'].str.contains("Toronto")].reset_index(drop = True)
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


#### Having multiple neighborhoods per row will create issues with the analysis so I will only use the first Nieghborhood in the analysis

In [118]:
#Creating a separate dataframe by splitting the Neighborhood column
df_split= df_toronto.Neighborhood.str.split(",", expand = True)
#Adding a prefix to the column names
df_split = df_split.add_prefix('Neighborhood_')
df_split.head()

,Neighborhood_0,Neighborhood_1,Neighborhood_2,Neighborhood_3,Neighborhood_4,Neighborhood_5,Neighborhood_6
0,Regent Park,Harbourfront,None,None,None,None,None
1,Queen's Park,Ontario Provincial Government,None,None,None,None,None
2,Garden District,Ryerson,None,None,None,None,None
3,St. James Town,None,None,None,None,None,None
4,The Beaches,None,None,None,None,None,None


In [120]:
#Only keeping the first listed neighborood
df_split = df_split[["Neighborhood_0"]]
df_split.reset_index(inplace = True, drop = True)
df_split.head()

,Neighborhood_0
0,Regent Park
1,Queen's Park
2,Garden District
3,St. James Town
4,The Beaches


In [121]:
#Concatenating the two dataframes together and dropping the old neighborhood column
df_clean = pd.concat([df_toronto, df_split], axis = 1)
df_clean = df_clean.drop(['Neighborhood'], axis = 1)
df_clean.head()

,PostalCode,Borough,Latitude,Longitude,Neighborhood_0
0,M5A,Downtown Toronto,43.654260,-79.360636,Regent Park
1,M7A,Downtown Toronto,43.662301,-79.389494,Queen's Park
2,M5B,Downtown Toronto,43.657162,-79.378937,Garden District
3,M5C,Downtown Toronto,43.651494,-79.375418,St. James Town
4,M4E,East Toronto,43.676357,-79.293031,The Beaches


In [122]:
#renaming the Neighborhood column to remove the prefix
df_clean.rename(columns = {'Neighborhood_0': 'Neighborhood'}, inplace = True)
df_clean.head()

,PostalCode,Borough,Latitude,Longitude,Neighborhood
0,M5A,Downtown Toronto,43.654260,-79.360636,Regent Park
1,M7A,Downtown Toronto,43.662301,-79.389494,Queen's Park
2,M5B,Downtown Toronto,43.657162,-79.378937,Garden District
3,M5C,Downtown Toronto,43.651494,-79.375418,St. James Town
4,M4E,East Toronto,43.676357,-79.293031,The Beaches


In [123]:
#renaming the dataframe for cleanliness
df_toronto = df_clean

In [124]:
#Creating a map of the Neighborhoods
#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

In [125]:
#google searching for the Toronto Neighborhood Coordinates
latitude = 43.6529
longitude = -79.3849

In [154]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [155]:
# The code was removed by Watson Studio for sharing.

In [156]:
#Exploring the first neighborhood

df_toronto.loc[0, 'Neighborhood']

'Regent Park '

#### Now, let's get the top 50 venues that are in Regent Park within a radius of 500 meters.

In [157]:
neighborhood_name = df_toronto.loc[0, 'Neighborhood']
neighborhood_latitude = df_toronto.loc[0, 'Latitude']
neighborhood_longitude = df_toronto.loc[0, 'Longitude']

LIMIT = 50 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=HSNVL3XPEP1ZEFL5PKKJDKHOFMSNBCM5WQ0JJL51RY5AV3GG&client_secret=GGRC30CTORKYNVWCDJOTTTUIPUJYYPBSBNEABBWXU13OJCOR&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=50'

In [158]:
#Sending the GET request to examine the results
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e8337497828ae001b96e165'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 46,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
 

In [159]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [160]:
#Cleaning Json and putting into pandas DF structure
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149


In [161]:
#Creating a function to repeat the same for all neighborhoods in our Toronto dataset
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [162]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )


Regent Park 
Queen's Park 
Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond 
Dufferin 
Harbourfront East 
Little Portugal 
The Danforth West 
Toronto Dominion Centre 
Brockton 
India Bazaar 
Commerce Court 
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park 
North Toronto West
The Annex 
Parkdale 
Davisville
University of Toronto 
Runnymede 
Moore Park 
Kensington Market 
Summerhill West 
CN Tower 
Rosedale
Stn A PO Boxes
St. James Town 
First Canadian Place 
Church and Wellesley
Business reply mail Processing CentrE


In [163]:
#Sizing and getting a view of the data
print(toronto_venues.shape)
toronto_venues.head()

(1200, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Regent Park,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Regent Park,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Regent Park,43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,Regent Park,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Regent Park,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [164]:
#Seeing how many venues we get per neighborhood
toronto_venues.groupby('Neighborhood').count().head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,50,50,50,50,50,50
Brockton,23,23,23,23,23,23
Business reply mail Processing CentrE,16,16,16,16,16,16
CN Tower,16,16,16,16,16,16
Central Bay Street,50,50,50,50,50,50


#### Analysing each neighborhood

In [165]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Market,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Regent Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Regent Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [166]:
#Grouping the rows by neighborhood and taking the mean of the frequency of the occurance
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Market,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.00,0.0,0.0,0.02,0.0,0.0,0.0000,0.02,0.0,0.02,0.040000,0.0,0.000000,0.0,0.02,0.02,0.04,0.0,0.0,0.02,0.0000,0.00,0.0000,0.0,0.020000,0.0000,0.00,0.00,0.000000,0.0,0.0,0.040000,0.0,0.0,0.0,0.0,0.04,0.00,0.0,0.0,0.000000,0.0,0.04,0.100000,0.0,0.0,0.0,0.0,0.0,0.0,0.02,0.0000,0.02,0.000000,0.0,0.0,0.02,0.0,0.0,0.0,0.0,0.02,0.00,0.02,0.00,0.0,0.0,0.0,0.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,0.0400,0.0000,0.0,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.02,0.02,0.0,0.0,0.0,0.000000,0.0,0.0000,0.0000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.02,0.02,0.000000,0.000000,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.02,0.0,0.00,0.0,0.0,0.000000,0.000000,0.02,0.02,0.0,0.0,0.0,0.0,0.0,0.0000,0.02,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.02,0.0,0.0,0.000000,0.0,0.00,0.0,0.0200,0.000000,0.000000,0.0,0.0,0.0000,0.0000,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0000,0.0000,0.040000,0.0,0.0,0.0,0.0,0.00,0.0,0.0000,0.04,0.0,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0

In [167]:
#Printing each neighborhood along with the 5 most common venues
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.10
1                Café  0.04
2         Cheese Shop  0.04
3            Beer Bar  0.04
4  Seafood Restaurant  0.04


----Brockton ----
            venue  freq
0            Café  0.13
1  Breakfast Spot  0.09
2     Coffee Shop  0.09
3     Yoga Studio  0.04
4   Grocery Store  0.04


----Business reply mail Processing CentrE----
              venue  freq
0               Spa  0.06
1     Auto Workshop  0.06
2       Pizza Place  0.06
3              Park  0.06
4  Recording Studio  0.06


----CN Tower ----
                 venue  freq
0       Airport Lounge  0.12
1     Airport Terminal  0.12
2                Plane  0.06
3  Rental Car Location  0.06
4   Airport Food Court  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.18
1  Italian Restaurant  0.06
2     Bubble Tea Shop  0.04
3                 Spa  0.04
4        Burger Joint  0.04


----Christie----
           venue  freq
0  Grocery 

In [168]:
#Writing a function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [169]:
#Creating a new DF to display the top venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Café,Restaurant,Beer Bar,Farmers Market,Seafood Restaurant,Breakfast Spot
1,Brockton,Café,Coffee Shop,Breakfast Spot,Yoga Studio,Bakery,Performing Arts Venue,Pet Store,Convenience Store,Climbing Gym,Restaurant
2,Business reply mail Processing CentrE,Garden,Comic Shop,Fast Food Restaurant,Recording Studio,Restaurant,Auto Workshop,Burrito Place,Gym / Fitness Center,Spa,Pizza Place
3,CN Tower,Airport Lounge,Airport Terminal,Coffee Shop,Harbor / Marina,Plane,Rental Car Location,Sculpture Garden,Boat or Ferry,Bar,Boutique
4,Central Bay Street,Coffee Shop,Italian Restaurant,Middle Eastern Restaurant,Bubble Tea Shop,Burger Joint,Japanese Restaurant,Thai Restaurant,Spa,Bookstore,Sandwich Place


#### Now that our data is in a workable format, we'll start our clustering

In [170]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [171]:
#creating anew dataframe that includes cluster and the top 10 venues per neighborhood
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Latitude,Longitude,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,43.654260,-79.360636,Regent Park,0,Coffee Shop,Pub,Park,Mexican Restaurant,Breakfast Spot,Restaurant,Café,Bakery,Theater,Shoe Store
1,M7A,Downtown Toronto,43.662301,-79.389494,Queen's Park,0,Coffee Shop,Wings Joint,Creperie,Nightclub,Mexican Restaurant,Juice Bar,Italian Restaurant,Hobby Shop,Gym,Fried Chicken Joint
2,M5B,Downtown Toronto,43.657162,-79.378937,Garden District,0,Coffee Shop,Café,Theater,Ramen Restaurant,Clothing Store,Restaurant,Cosmetics Shop,Bookstore,Pizza Place,Spa
3,M5C,Downtown Toronto,43.651494,-79.375418,St. James Town,0,Coffee Shop,Italian Restaurant,Japanese Restaurant,Café,BBQ Joint,Beer Bar,Thai Restaurant,Cocktail Bar,Restaurant,Gastropub
4,M4E,East Toronto,43.676357,-79.293031,The Beaches,0,Trail,Pub,Health Food Store,Wings Joint,Cupcake Shop,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


In [172]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Examining Clusters

In [174]:
#Cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Pub,Park,Mexican Restaurant,Breakfast Spot,Restaurant,Café,Bakery,Theater,Shoe Store
1,Downtown Toronto,0,Coffee Shop,Wings Joint,Creperie,Nightclub,Mexican Restaurant,Juice Bar,Italian Restaurant,Hobby Shop,Gym,Fried Chicken Joint
2,Downtown Toronto,0,Coffee Shop,Café,Theater,Ramen Restaurant,Clothing Store,Restaurant,Cosmetics Shop,Bookstore,Pizza Place,Spa
3,Downtown Toronto,0,Coffee Shop,Italian Restaurant,Japanese Restaurant,Café,BBQ Joint,Beer Bar,Thai Restaurant,Cocktail Bar,Restaurant,Gastropub
4,East Toronto,0,Trail,Pub,Health Food Store,Wings Joint,Cupcake Shop,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store
5,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Café,Restaurant,Beer Bar,Farmers Market,Seafood Restaurant,Breakfast Spot
6,Downtown Toronto,0,Coffee Shop,Italian Restaurant,Middle Eastern Restaurant,Bubble Tea Shop,Burger Joint,Japanese Restaurant,Thai Restaurant,Spa,Bookstore,Sandwich Place
7,Downtown Toronto,0,Grocery Store,Café,Park,Baby Store,Coffee Shop,Nightclub,Candy Store,Restaurant,Diner,Italian Restaurant
8,Downtown Toronto,0,Café,Seafood Restaurant,American Restaurant,Bar,Steakhouse,Restaurant,Asian Restaurant,Gastropub,Hotel,Pizza Place
9,West Toronto,0,Pharmacy,Bakery,Gym / Fitness Center,Music Venue,Pool,Middle Eastern Restaurant,Café,Brewery,Fast Food Restaurant,Bar


In [175]:
#Cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,Central Toronto,1,Tennis Court,Cupcake Shop,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop


In [176]:
#Cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,2,Garden,Music Venue,Wings Joint,Dance Studio,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


In [177]:
#Cluster 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Central Toronto,3,Jewelry Store,Trail,Sushi Restaurant,Mexican Restaurant,Dance Studio,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


In [178]:
#Cluster 5
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,4,Photography Studio,Park,Swim School,Bus Line,Dance Studio,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store
33,Downtown Toronto,4,Park,Playground,Trail,Wings Joint,Cupcake Shop,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


### Looking at the clusters, we have many the most homogeneity in cluster 1 meaning that those boroughs are probably the most similar. If you are looking for something a bit different, it seems that Boroughs in cluster 2-5 would be something to look into.